In [ ]:
#   Copyright 2025 UKRI-STFC

#   Licensed under the Apache License, Version 2.0 (the "License");
#   you may not use this file except in compliance with the License.
#   You may obtain a copy of the License at

#   http://www.apache.org/licenses/LICENSE-2.0

#   Unless required by applicable law or agreed to in writing, software
#   distributed under the License is distributed on an "AS IS" BASIS,
#   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#   See the License for the specific language governing permissions and
#   limitations under the License.
#
# Authors:
# Franck Vidal (URKI-STFC)

# RXSolutionsReader CT Demo

## Data format: RX Solutions

The data is in the format used by devices made by [RX Solutions](https://www.rx-solutions.com/en). The projections are saved in TIFF files. They are flatfield corrected using 16-bit unsigned integers. Metadata is saved in two different files, an XML file that can be used with orbital geometries, and a CSV file that can be used with flexible geometries.

## CIL Version

This notebook was developed using CIL v25.0.0

## Dataset
The data is available from Zenodo: https://doi.org/10.5281/zenodo.15835893

They are XCT datasets of a part of a lab tensile machine for in situ stress in scanning electron microscopes (SEM). 
They were acquired with the [Dual Tube High Energy (DTHE)](https://www.rx-solutions.com/en/blog/126/dthe-technology) microCT platform developed by [RX Solutions](https://www.rx-solutions.com/en) for the [MATEIS Laboratory](https://mateis.insa-lyon.fr/en) of [INSA-Lyon](https://www.insa-lyon.fr/en/). Two The datasets include projections and reconstructions for:

- Tube voltage: 160 kV; Filtration: None; Amperage: 200uA; exposure: 0.167s; SDD: 807.249mm; SOD: 306.416mm
- Tube voltage: 160 kV; Filtration: 0.4mm of Cu; Amperage: 530uA; exposure: 0.167s; SDD: 807.249mm; SOD: 306.416mm

We acknowledge [Taith](https://www.taith.wales/), Wales' international learning exchange programme, for two travel grants to Lyon, France.

Update this filepath to where you have saved the dataset:

In [ ]:
XML_file_path = '/mnt/c/Users/res52584/DATA/DTHE/ZrO2-Cu-1mm-10umvx/unireconstruction.xml'
CSV_file_path = '/mnt/c/Users/res52584/DATA/DTHE/ZrO2-Cu-1mm-10umvx/geom.csv'

In [ ]:
import numpy as np
import gc

from cil.utilities.display import show2D, show_geometry, show_system_positions
from cil.processors import TransmissionAbsorptionConverter, Slicer, CentreOfRotationCorrector
from cil.framework import ImageGeometry
from cil.recon import FDK
from cil.plugins.astra import FBP
from cil.utilities.jupyter import islicer, link_islicer

from readers.RXSolutionsDataReader import RXSolutionsDataReader

# Using an orbital geometry

## Loading Geometry

If you don't know what acquisition geometry (`acq_geom`) is used, the code below will show the geometry regardless of the type of geometry.

```python
if acq_geom.geom_type != "CONE_FLEX":
    show_geometry(acq_geom)
else:
    show_system_positions(acq_geom)
```

In [ ]:
reader = RXSolutionsDataReader(XML_file_path)

In [ ]:
orbital_acq_geom = reader.get_geometry()

In [ ]:
show_geometry(orbital_acq_geom)

In [ ]:
print(orbital_acq_geom)

## Loading Projections

In [ ]:
orbital_acq_data = reader.read()

In [ ]:
show2D(orbital_acq_data, origin='upper-left');

## Pre-processing and Reconstruction

In [ ]:
orbital_data_exp = TransmissionAbsorptionConverter()(orbital_acq_data)

In [ ]:
# data_crop = Slicer(roi={'vertical': (250, 450, 1)})(orbital_data_exp)

In [ ]:
orbital_image_geometry = orbital_data_exp.geometry.get_ImageGeometry()
print(orbital_image_geometry)

In [ ]:
orbital_recon = FDK(orbital_data_exp, orbital_image_geometry).run()

## Visualise the reconstruction

In [ ]:
islicer(orbital_recon)

## Release memory

In [ ]:
# del data_crop
del orbital_data_exp
del orbital_acq_data
del reader

gc.collect();

# Using a per-projection geometry

## Loading Geometry

In [ ]:
reader = RXSolutionsDataReader(CSV_file_path)

In [ ]:
flexible_acq_geom = reader.get_geometry()

In [ ]:
show_system_positions(flexible_acq_geom)

In [ ]:
print(flexible_acq_geom)

## Loading Projections

In [ ]:
flexible_acq_data = reader.read()

In [ ]:
show2D(flexible_acq_data, origin='upper-left');

# Pre-processing and Reconstruction

In [ ]:
# Prepare the data for Astra
flexible_acq_data.reorder(order='astra');

flexible_data_exp = TransmissionAbsorptionConverter()(flexible_acq_data)

In [ ]:
# Use the system magnification to compute the voxel size
mag = flexible_data_exp.geometry.magnification
mean_mag = np.mean(mag)
print("Mean magnification: ", mean_mag)

voxel_size_xy = flexible_data_exp.geometry.config.panel.pixel_size[0] / mean_mag
voxel_size_z = flexible_data_exp.geometry.config.panel.pixel_size[1] / mean_mag

# Create an image geometry
num_voxel_xy = int(np.ceil(flexible_data_exp.geometry.config.panel.num_pixels[0]))
num_voxel_z = int(np.ceil(flexible_data_exp.geometry.config.panel.num_pixels[1]))

image_geometry = ImageGeometry(num_voxel_xy, num_voxel_xy, num_voxel_z, voxel_size_xy, voxel_size_xy, voxel_size_z)
# image_geometry.center_z = -200
print(image_geometry)

In [ ]:
# Reconstruct using FDK
# Instantiate the reconstruction algorithm
fdk = FBP(image_geometry, flexible_data_exp.geometry)
fdk.set_input(flexible_data_exp)

# Perform the actual CT reconstruction
flexible_recon = fdk.get_output()

## Visualise the reconstruction

In [ ]:
islicer(flexible_recon)

# Compare the two reconstructions

In [ ]:
show2D([orbital_recon, flexible_recon], slice_list=(('horizontal_y',350)))

In [ ]:
link_islicer(islicer(orbital_recon), islicer(flexible_recon))